In [ ]:
from config import Color
from helpers import clean, local2global
from helpers.change_coordinates import Offset
from plan import Plan
from simulators import QGC, ConfigGazebo, Gazebo, Simulator

clean()

## Create Plans

In [ ]:
gcses = [
    ("blue 🟦", Color.BLUE),
    ("green 🟩", Color.GREEN),
    ("yellow 🟨", Color.YELLOW),
    # ("orange 🟧", Color.ORANGE),
    # ("red 🟥", Color.RED),
]
n_uavs_per_gcs = 5
side_len = 5
altitude = 5

n_gcs = len(gcses)
n_vehicles = n_gcs * n_uavs_per_gcs    
offsets:list[Offset] = [
    (i * 10 * side_len, j * 3 * side_len, 0, 0) for i in range(n_gcs) for j in range(n_uavs_per_gcs)
]

local_paths = [Plan.create_square_path(side_len=side_len, alt=altitude) for i in range(n_vehicles)]
plans = [Plan.basic(wps=path, wp_margin=0.5) for path in local_paths]

## Visualization Parameters

In [ ]:
homes = [offset[:3] for offset in offsets]
global_paths = [local2global(path, home) for path, home in zip(local_paths, homes)]
models = [('iris', gcses[i][1]) for i in range(n_gcs) for j in range(n_uavs_per_gcs)]
markers = [ConfigGazebo.create_trajectory_from_array(array=global_paths[i],color=models[i][1]) for i in range(n_vehicles)]
Gazebo.plot_3d_interactive(markers=markers, title="Simulation Markers")

## Choose Simulator

In [ ]:
simulator = Simulator(offsets=offsets, plans=plans)

simulator = QGC(offsets=offsets, plans=plans, origin=(-35.3633245, 149.1652241, 0, 0))

gazebo_config = ConfigGazebo(world_path="simulators/gazebo/worlds/runway3.world",models=models,markers = markers)
simulator = Gazebo(offsets=offsets,plans=plans,config=gazebo_config, visible_terminals=False)

## Launch Simulator

In [ ]:
sysids = list(range(1,n_vehicles + 1))
gcs_sysids = {gcses[i][0]: sysids[i * n_uavs_per_gcs:(i + 1) * n_uavs_per_gcs] for i in range(n_gcs)}
orac = simulator.launch(gcs_sysids)

## Execute Plans

In [ ]:
while len(orac.conns):
    for sysid in list(orac.conns.keys()):
        if orac.is_plan_done(sysid):
            orac.remove(sysid)

In [ ]:
import pickle

import numpy as np
from numpy.typing import NDArray

colors=[Color.BLUE,Color.GREEN,Color.YELLOW] 
for gcs_name,color in zip(gcs_sysids,colors):
    with open(f"trajectories_{gcs_name}.pkl", "rb") as f:
        trajs = pickle.load(f)
    paths:list[NDArray[np.float64]] = []
    for sysid,path in trajs.items():
        paths.append(np.array([x for x in path if x is not None]))
    markers = [ConfigGazebo.create_trajectory_from_array(array=path,color=color) for path in paths]
    Gazebo.plot_3d_interactive(markers=markers,title="Simulation Waypoints",frames=(0.2, 0.2, 0.6),ground=-0.05)